In [102]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import webbrowser

Liste des lieux et url 

In [103]:

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'hotel-de-ville',
'pont-alexandre-iii',
'trocadero', 
'la-concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

for url in urls:
    webbrowser.get('safari').open(url)

In [95]:
def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Recherche des lignes contenant les informations spécifiques
        specific_lines = []
        for paragraph in soup.find_all('p'):
            if 'Capacité' in paragraph.text or 'Capacité court Philippe-Chatrier' in paragraph.text or 'Volleyball' in paragraph.text or 'Pour la gymnastique rythmique' in paragraph.text:
                specific_lines.append(paragraph.text.strip())
        return specific_lines
    return None

# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Informations'])

display(df)

,Lieu,Informations
0,stade-tour-eiffel,[Capacité : 12 860 spectateurs]
1,arena-champ-de-mars,[Capacité : 8 356 places]
2,grand-palais,[Capacité : 8 000 personnes pendant les Jeux O...
3,invalides,[Capacité : 8 000 places]
4,hotel-de-ville,[]
5,pont-alexandre-iii,[Capacité : 1 000 personnes]
6,trocadero,[Capacité : 3 349 places]
7,la-concorde,None
8,stade-roland-garros,[Capacité court Philippe-Chatrier : 14 962 pla...
9,arena-paris-sud,[]


Compléter pour les valeurs manquantes : Hôtel de ville

In [96]:
#Compléter pour les valeurs manquantes : hôtel de ville
df.iloc[4, 1] = 45000

Compléter pour les valeurs manquantes : La Concorde

In [97]:
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[7, 1]=texte_places.strip()

/var/folders/71/d9rlmw9s3tvck7609v3sg_2w0000gn/T/ipykernel_63691/636666933.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texte_places = soup.find(text=lambda text: text and "places" in text)


Compléter pour les valeurs manquantes : Arena Paris Sud

In [98]:
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[9, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

display(df)

/var/folders/71/d9rlmw9s3tvck7609v3sg_2w0000gn/T/ipykernel_63691/2761411153.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)


,Lieu,Informations
0,stade-tour-eiffel,[Capacité : 12 860 spectateurs]
1,arena-champ-de-mars,[Capacité : 8 356 places]
2,grand-palais,[Capacité : 8 000 personnes pendant les Jeux O...
3,invalides,[Capacité : 8 000 places]
4,hotel-de-ville,45000
5,pont-alexandre-iii,[Capacité : 1 000 personnes]
6,trocadero,[Capacité : 3 349 places]
7,la-concorde,Des tribunes de 35 000 places seront alors age...
8,stade-roland-garros,[Capacité court Philippe-Chatrier : 14 962 pla...
9,arena-paris-sud,9 000 (maximum)


Extraire uniquement les nombres 

In [99]:
# Fonction pour extraire uniquement les chiffres de la colonne 'Informations'
def extraire_chiffres(text):
    chiffres = re.sub(r'\D', '', str(text))  # Utilisation d'une regex pour garder uniquement les chiffres
    return chiffres

# Appliquer la fonction à la colonne 'Informations'
df['Informations'] = df['Informations'].apply(extraire_chiffres)

display(df)

,Lieu,Informations
0,stade-tour-eiffel,12860
1,arena-champ-de-mars,2028202356202
2,grand-palais,80006500
3,invalides,08000
4,hotel-de-ville,45000
5,pont-alexandre-iii,1000
6,trocadero,2023202349202
7,la-concorde,35000
8,stade-roland-garros,020214962
9,arena-paris-sud,9000


In [100]:
# Appliquer une fonction lambda pour chaque élément de la colonne 'Informations'
df['Informations'] = df['Informations'].apply(lambda x: re.sub(r'\D', '', str(x)))

# Afficher le DataFrame mis à jour
display(df)

,Lieu,Informations
0,stade-tour-eiffel,12860
1,arena-champ-de-mars,2028202356202
2,grand-palais,80006500
3,invalides,08000
4,hotel-de-ville,45000
5,pont-alexandre-iii,1000
6,trocadero,2023202349202
7,la-concorde,35000
8,stade-roland-garros,020214962
9,arena-paris-sud,9000
